<a href="https://colab.research.google.com/github/atlawand/Bioinformatics/blob/master/Copy_of_TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


In [ ]:
import numpy as np
import torch

In [ ]:
#Input (temp, rainfall, humidity)
inputs = np.array([[73,67,43],
                   [91,88,64],
                   [87,134,58],
                   [102,43,37],
                   [69,96,70]], dtype='float32')

In [ ]:
#Target (apples, oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]], dtype= 'float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
#Intialize weights and biases
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.9050,  0.5896,  0.0866],
        [ 1.0580, -0.0417, -0.3401]], requires_grad=True)
tensor([-1.4877, -1.6675], requires_grad=True)


In [ ]:
def model(x):
  return x @ w.t() + b

In [ ]:
#Generate Predictions
preds = model(inputs)
print(preds)

tensor([[107.8060,  58.1464],
        [138.2964,  69.1722],
        [161.2797,  65.0634],
        [119.3809,  91.8683],
        [123.6225,  43.5231]], grad_fn=<AddBackward0>)


In [ ]:
# compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
diff = preds - targets
torch.sum(diff*diff)/diff.numel()

tensor(3213.8833, grad_fn=<DivBackward0>)

In [ ]:
#MSE (mean square error)
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff*diff)/diff.numel()

In [ ]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(3213.8833, grad_fn=<DivBackward0>)


In [ ]:
#Compute Gradients
loss.backward()

In [ ]:
print(w)
print(w.grad)

tensor([[ 0.9050,  0.5896,  0.0866],
        [ 1.0580, -0.0417, -0.3401]], requires_grad=True)
tensor([[ 4805.9888,  4069.1392,  2678.7026],
        [-1856.6931, -3516.9980, -1948.0527]])


In [ ]:
print(b)
print(b.grad)

tensor([-1.4877, -1.6675], requires_grad=True)
tensor([ 53.8771, -26.4453])


In [ ]:
print(w)
w.grad

tensor([[ 0.9050,  0.5896,  0.0866],
        [ 1.0580, -0.0417, -0.3401]], requires_grad=True)


tensor([[ 4805.9888,  4069.1392,  2678.7026],
        [-1856.6931, -3516.9980, -1948.0527]])

In [ ]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

In [ ]:
w, b

(tensor([[ 0.8089,  0.5082,  0.0330],
         [ 1.0951,  0.0287, -0.3011]], requires_grad=True),
 tensor([-1.4888, -1.6669], requires_grad=True))

In [ ]:
print(w)
w - w.grad * 1e-5

tensor([[ 0.8089,  0.5082,  0.0330],
        [ 1.0951,  0.0287, -0.3011]], requires_grad=True)


tensor([[ 0.7608,  0.4676,  0.0062],
        [ 1.1137,  0.0638, -0.2817]], grad_fn=<SubBackward0>)

In [ ]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(2117.1353, grad_fn=<DivBackward0>)


In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x
